In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
dis = 'drive/My Drive/data_final/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv(dis + 'data1.csv')

##Count Small Group

In [ ]:
agg_features = data.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()

counter_df_train = data.groupby(['client_dk','small_group'])['amount'].count()
cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_'+str(i) for i in cat_counts_train.columns]

In [ ]:
train_target = pd.read_csv(dis + 'train_target.csv')
train_target['is_train'] = 1

train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')
train_target = train_target.fillna(0)
for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_target[str(i)] = train_target[str(i)].astype(int)
 
train_target['is_train'] = train_target['is_train'].astype(int)

train_target = train_target.set_index('client_dk')

## Small Group Rate

In [ ]:
agg_features = agg_features.reindex(train_target.index)

In [ ]:
for i in cat_counts_train.columns:
  train_target[i + '_rate'] = train_target[i]/agg_features['count']

In [ ]:
train_target.to_csv(dis + 'predata6.csv')

## Small Group Sum

In [ ]:
counter_df_train = data.groupby(['client_dk','small_group'])['amount'].sum()
cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_sum_'+str(i) for i in cat_counts_train.columns]

In [ ]:
train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

In [ ]:
train_target.to_csv(dis + 'predata_sum.csv')

## Small group mean

In [ ]:
counter_df_train = data.groupby(['client_dk','small_group'])['amount'].mean()
cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_mean_'+str(i) for i in cat_counts_train.columns]

In [ ]:
train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

In [ ]:
train_target.to_csv(dis + 'predata_mean.csv')

## Small Group std

In [ ]:
counter_df_train = data.groupby(['client_dk','small_group'])['amount'].std()
cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_std_'+str(i) for i in cat_counts_train.columns]

In [ ]:
train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

In [ ]:
train_target.to_csv(dis + 'predata_std.csv')

## Last buy

In [ ]:
train_target = pd.read_csv(dis + 'predata_std.csv', index_col = 'client_dk')

In [ ]:
train_target = train_target.reindex(sorted(train_target.index))

In [ ]:
train_target

,27,32,41,45,67,73,81,88,is_train,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,...,small_group_std_156,small_group_std_157,small_group_std_158,small_group_std_159,small_group_std_160,small_group_std_161,small_group_std_162,small_group_std_163,small_group_std_164,small_group_std_165,small_group_std_166,small_group_std_167,small_group_std_168,small_group_std_169,small_group_std_170,small_group_std_171,small_group_std_172,small_group_std_173,small_group_std_174,small_group_std_175,small_group_std_176,small_group_std_177,small_group_std_178,small_group_std_179,small_group_std_180,small_group_std_181,small_group_std_182,small_group_std_183,small_group_std_184,small_group_std_185,small_group_std_186,small_group_std_187,small_group_std_188,small_group_std_189,small_group_std_190,small_group_std_191,small_group_std_192,small_group_std_193,small_group_std_195,small_group_std_196
client_dk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1,1,9.0,0.0,27.0,8.0,89.0,119.0,309.0,22.0,0.0,12.0,0.0,15.0,15.0,14.0,15.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,13.0,15.0,0.0,1.0,2.0,0.0,2.0,0.0,24.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,13.0,0.0,314.0,3.0,164.0,194.0,182.0,20.0,0.0,5.0,5.0,86.0,2.0,0.0,9.0,0.0,1.0,0.0,66.0,0.0,1.0,0.0,13.0,19.0,2.0,1.0,1.0,3.0,3.0,0.0,2.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0,0,0,0,0,1,27.0,617.0,196.0,46.0,37.0,114.0,191.0,0.0,1.0,4.0,9.0,79.0,2.0,11.0,3.0,2.0,1.0,0.0,9.0,0.0,1.0,1.0,11.0,0.0,0.0,0.0,0.0,10.0,1.0,2.0,2.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,1,0,0,0,0,1,16.0,0.0,1.0,19.0,28.0,174.0,320.0,15.0,0.0,23.0,14.0,20.0,23.0,37.0,18.0,3.0,0.0,5.0,0.0,1.0,24.0,0.0,42.0,0.0,1.0,2.0,9.0,1.0,2.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,1,21.0,0.0,320.0,50.0,25.0,308.0,105.0,45.0,0.0,46.0,3.0,74.0,16.0,3.0,41.0,18.0,4.0,1.0,20.0,1.0,8.0,0.0,34.0,45.0,0.0,2.0,2.0,2.0,1.0,0.0,2.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,1,0,0,0,0,1,0.0,0.0,3.0,1.0,0.0,42.0,27.0,0.0,0.0,36.0,55.0,187.0,6.0,0.0,34.0,0.0,0.0,1.0,0.0,0.0,16.0,0.0,19.0,0.0,1.0,8.0,1.0,17.0,38.0,4.0,0.0,...,0.0,134.725277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0,1,0,0,0,0,0,0,1,11.0,0.0,0.0,12.0,12.0,170.0,148.0,0.0,0.0,17.0,10.0,361.0,23.0,0.0,11.0,2.0,1.0,5.0,1.0,3.0,7.0,0.0,30.0,3.0,0.0,3.0,2.0,13.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
import progressbar
import time
bar = progressbar.ProgressBar().start() # Создаём новый progress bar

d = [[] for i in range(199)]
for i in range(199):
  c = data.loc[data['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
  for j in range(50000):
    if j in c.index:
      c0 = list(c[j].index)
      d[i].append((387 - c0[-1]))
    else:
      d[i].append(-1)
  bar.update((i + 1)) 

| |      #                                          | 199 Elapsed Time: 0:14:42

In [ ]:
for i in range(199):
  train_target['last_buy_small_group_' + str(i)] = d[i]

In [ ]:
train_target.to_csv(dis + 'predata_last_buy.csv')

## Delta days

In [ ]:
import progressbar
import time
bar = progressbar.ProgressBar().start() # Создаём новый progress bar

d = [[] for i in range(199)]
for i in range(199):
  c = data.loc[data['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
  for j in range(50000):
    if j in c.index:
      c0 = list(c[j].index)
      if len(c0) > 1:
        d[i].append((c0[-1] - c0[0])/(len(c0) - 1))
      else:
        d[i].append(387)
    else:
      d[i].append(-1)
  bar.update((i + 1)) 

| |                                    #            | 199 Elapsed Time: 0:14:55

In [ ]:
for i in range(199):
  train_target['delta_day_small_group_' + str(i)] = d[i]

In [ ]:
train_target.to_csv(dis + 'predata_delta_day.csv')

## Weekly Friction

In [ ]:
train_target = pd.read_csv(dis + 'predata_delta_day.csv', index_col = 'client_dk')

In [ ]:
mas = [27, 32,	41,	45,	67,	73,	81,	88]

In [ ]:
def GetWeekly(data, l):
    td = data.loc[(data['trans_date'] >= l) & (data['trans_date'] < l + 7)]

    counter_df_train = td.groupby(['client_dk','small_group'])['amount'].count()
    cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    cat_counts_train = cat_counts_train.fillna(0)
    cat_counts_train.columns = [i for i in cat_counts_train.columns]

    for i in cat_counts_train.columns:
      cat_counts_train[i] = cat_counts_train[i].astype(int)

    return cat_counts_train[[27, 32,	41,	45,	67,	73,	81,	88]]

In [ ]:
app = pd.DataFrame(train_target.index).set_index('client_dk')
app = app.merge(GetWeekly(data, 0), how = 'outer', on = 'client_dk')

import progressbar
import time
bar = progressbar.ProgressBar().start()

for i in range(1, 51):
  app += GetWeekly(data, i*7)
  app = app.fillna(0)
  bar.update(i)

\ |                                            #     | 50 Elapsed Time: 0:02:05

In [ ]:
app /= 50

In [ ]:
app.columns = ['friction_' + str(i) for i in mas]

In [ ]:
train_target = train_target.merge(app, how = 'outer', on = 'client_dk')

In [ ]:
train_target.to_csv(dis + 'predata_weekly.csv')

## Trans date

In [ ]:
def Standart(p):
  for i in p.columns:
    p[i][p[i] > 0] = 1
  for i in p.columns:
    p[i] = p[i].astype(int)

In [ ]:
df_filter = data['small_group'].isin([27, 32,	41,	45,	67,	73,	81,	88])
dl = data[df_filter]

p = dl.groupby(['client_dk', 'trans_date'])['amount'].count().reset_index().pivot(index = 'client_dk',
                                                                                                               columns = 'trans_date',
                                                                                                               values = 'amount').fillna(0)

p.columns = ['day_' + str(i) for i in p.columns]

In [ ]:
Standart(p)

In [ ]:
train_target = train_target.merge(p, how = 'outer', on = 'client_dk').fillna(0)

In [ ]:
for i in p.columns:
  train_target[i] = train_target[i].astype(int)

In [ ]:
train_target.to_csv(dis + 'predata_daily.csv')

## Previous buy

In [ ]:
train_target = pd.read_csv(dis + 'predata_delta_day.csv', index_col = 'client_dk')

In [ ]:
train_b = train_target.drop(['last_buy_small_group_' + str(i) for i in range(199)], axis =  1)
train_b = train_b.drop(['delta_day_small_group_' + str(i) for i in range(199)], axis = 1)
train_b_exp = train_target[['delta_day_small_group_' + str(i) for i in range(199)]]

In [ ]:
import progressbar
import time
bar = progressbar.ProgressBar().start() # Создаём новый progress bar

data0 = data.loc[data['trans_date'] < 200]

d = [[] for i in range(199)]
for i in range(199):
  c = data0.loc[data0['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
  for j in range(50000):
    if j in c.index:
      c0 = list(c[j].index)
      d[i].append((200 - c0[-1]))
    else:
      d[i].append(-1)
  bar.update((i + 1)) 

| |     #                                           | 199 Elapsed Time: 0:08:59

In [ ]:
for i in range(199):
  train_b['last_buy_small_group_' + str(i)] = d[i]

In [ ]:
def GetWeekly(data, l):
    td = data.loc[(data['trans_date'] >= l) & (data['trans_date'] < l + 7)]

    counter_df_train = td.groupby(['client_dk','small_group'])['amount'].count()
    cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    cat_counts_train = cat_counts_train.fillna(0)
    cat_counts_train.columns = [i for i in cat_counts_train.columns]

    for i in cat_counts_train.columns:
      cat_counts_train[i] = cat_counts_train[i].astype(int)

    return cat_counts_train[[27, 32,	41,	45,	67,	73,	81,	88]]

In [ ]:
bc = GetWeekly(data, 200)
for i in [27, 32,	41,	45,	67,	73,	81,	88]:
  bc[i][bc[i] > 0] = 1

In [ ]:
for i in [27, 32,	41,	45,	67,	73,	81,	88]:
  train_b[str(i)] = bc[i]

In [ ]:
train_b = train_b.fillna(0)

In [ ]:
for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_b[str(i)] = train_b[str(i)].astype(int)

In [ ]:
train_b = train_b.merge(train_b_exp, on = 'client_dk', how='outer')

In [ ]:
train_b.index = list(map(lambda x: 'a' + str(x), train_b.index))

In [ ]:
train_target[['last_buy_small_group_' + str(i) for i in range(199)]]

,last_buy_small_group_0,last_buy_small_group_1,last_buy_small_group_2,last_buy_small_group_3,last_buy_small_group_4,last_buy_small_group_5,last_buy_small_group_6,last_buy_small_group_7,last_buy_small_group_8,last_buy_small_group_9,last_buy_small_group_10,last_buy_small_group_11,last_buy_small_group_12,last_buy_small_group_13,last_buy_small_group_14,last_buy_small_group_15,last_buy_small_group_16,last_buy_small_group_17,last_buy_small_group_18,last_buy_small_group_19,last_buy_small_group_20,last_buy_small_group_21,last_buy_small_group_22,last_buy_small_group_23,last_buy_small_group_24,last_buy_small_group_25,last_buy_small_group_26,last_buy_small_group_27,last_buy_small_group_28,last_buy_small_group_29,last_buy_small_group_30,last_buy_small_group_31,last_buy_small_group_32,last_buy_small_group_33,last_buy_small_group_34,last_buy_small_group_35,last_buy_small_group_36,last_buy_small_group_37,last_buy_small_group_38,last_buy_small_group_39,...,last_buy_small_group_159,last_buy_small_group_160,last_buy_small_group_161,last_buy_small_group_162,last_buy_small_group_163,last_buy_small_group_164,last_buy_small_group_165,last_buy_small_group_166,last_buy_small_group_167,last_buy_small_group_168,last_buy_small_group_169,last_buy_small_group_170,last_buy_small_group_171,last_buy_small_group_172,last_buy_small_group_173,last_buy_small_group_174,last_buy_small_group_175,last_buy_small_group_176,last_buy_small_group_177,last_buy_small_group_178,last_buy_small_group_179,last_buy_small_group_180,last_buy_small_group_181,last_buy_small_group_182,last_buy_small_group_183,last_buy_small_group_184,last_buy_small_group_185,last_buy_small_group_186,last_buy_small_group_187,last_buy_small_group_188,last_buy_small_group_189,last_buy_small_group_190,last_buy_small_group_191,last_buy_small_group_192,last_buy_small_group_193,last_buy_small_group_194,last_buy_small_group_195,last_buy_small_group_196,last_buy_small_group_197,last_buy_small_group_198
client_dk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,112,-1,3,64,1,2,2,13,-1,14,-1,23,19,71,0,76,-1,-1,293,-1,25,-1,4,15,-1,214,166,-1,332,-1,5,45,13,128,161,-1,-1,-1,-1,125,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,23,-1,0,107,1,2,1,135,-1,47,101,0,130,-1,104,-1,180,-1,3,-1,309,-1,43,50,119,344,343,101,243,-1,1,-1,79,-1,243,146,-1,45,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,49,1,3,7,11,3,0,-1,119,153,37,6,297,9,152,281,367,-1,1,-1,131,282,145,-1,-1,-1,-1,32,296,51,74,-1,11,63,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,8,-1,140,8,58,2,1,12,-1,11,18,41,2,2,1,42,-1,46,-1,119,12,-1,4,-1,128,160,6,282,181,-1,-1,71,5,-1,15,35,-1,83,-1,314,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,-1,4,0,11,0,0,7,-1,0,85,0,9,11,3,120,0,135,6,153,23,-1,0,5,-1,21,0,248,211,-1,122,0,91,-1,35,149,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-1,-1,152,373,-1,4,7,-1,-1,18,2,1,64,-1,3,-1,-1,249,-1,-1,11,-1,6,-1,191,3,248,52,17,155,-1,46,50,-1,151,17,17,166,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
49996,44,-1,-1,130,3,0,0,-1,-1,27,2,2,76,-1,119,182,256,127,159,125,100,-1,11,243,-1,84,98,29,-1,-1,-1,98,9,-1,111,118,-1,115,-1,-1,...,-1,-1,-1,-1,

In [ ]:
train_b[['last_buy_small_group_' + str(i) for i in range(199)]]

,last_buy_small_group_0,last_buy_small_group_1,last_buy_small_group_2,last_buy_small_group_3,last_buy_small_group_4,last_buy_small_group_5,last_buy_small_group_6,last_buy_small_group_7,last_buy_small_group_8,last_buy_small_group_9,last_buy_small_group_10,last_buy_small_group_11,last_buy_small_group_12,last_buy_small_group_13,last_buy_small_group_14,last_buy_small_group_15,last_buy_small_group_16,last_buy_small_group_17,last_buy_small_group_18,last_buy_small_group_19,last_buy_small_group_20,last_buy_small_group_21,last_buy_small_group_22,last_buy_small_group_23,last_buy_small_group_24,last_buy_small_group_25,last_buy_small_group_26,last_buy_small_group_27,last_buy_small_group_28,last_buy_small_group_29,last_buy_small_group_30,last_buy_small_group_31,last_buy_small_group_32,last_buy_small_group_33,last_buy_small_group_34,last_buy_small_group_35,last_buy_small_group_36,last_buy_small_group_37,last_buy_small_group_38,last_buy_small_group_39,...,last_buy_small_group_159,last_buy_small_group_160,last_buy_small_group_161,last_buy_small_group_162,last_buy_small_group_163,last_buy_small_group_164,last_buy_small_group_165,last_buy_small_group_166,last_buy_small_group_167,last_buy_small_group_168,last_buy_small_group_169,last_buy_small_group_170,last_buy_small_group_171,last_buy_small_group_172,last_buy_small_group_173,last_buy_small_group_174,last_buy_small_group_175,last_buy_small_group_176,last_buy_small_group_177,last_buy_small_group_178,last_buy_small_group_179,last_buy_small_group_180,last_buy_small_group_181,last_buy_small_group_182,last_buy_small_group_183,last_buy_small_group_184,last_buy_small_group_185,last_buy_small_group_186,last_buy_small_group_187,last_buy_small_group_188,last_buy_small_group_189,last_buy_small_group_190,last_buy_small_group_191,last_buy_small_group_192,last_buy_small_group_193,last_buy_small_group_194,last_buy_small_group_195,last_buy_small_group_196,last_buy_small_group_197,last_buy_small_group_198
a0,68,-1,32,157,3,4,3,8,-1,86,-1,8,22,-1,27,-1,-1,-1,106,-1,-1,-1,33,7,-1,27,55,-1,145,-1,4,8,180,-1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
a1,24,-1,7,16,2,2,2,12,-1,13,4,1,174,-1,16,-1,-1,-1,1,-1,122,-1,4,16,143,157,156,27,56,-1,-1,-1,75,-1,56,33,-1,42,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
a2,4,4,1,5,5,9,1,-1,-1,38,166,9,110,-1,88,94,180,-1,110,-1,-1,95,7,-1,-1,-1,-1,10,109,166,-1,-1,-1,-1,-1,125,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
a3,10,-1,-1,39,7,1,1,11,-1,10,22,10,8,53,24,-1,-1,-1,-1,-1,13,-1,18,-1,-1,165,12,95,125,-1,-1,11,129,-1,101,30,-1,23,-1,127,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
a4,42,-1,1,43,-1,2,13,3,-1,20,148,3,20,-1,25,136,-1,-1,16,-1,65,-1,7,24,-1,-1,-1,61,24,-1,-1,33,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a49995,-1,-1,-1,186,-1,24,6,-1,-1,6,4,2,113,-1,36,-1,-1,62,-1,-1,4,-1,44,-1,4,53,61,66,38,18,-1,113,89,-1,-1,46,-1,39,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
a49996,54,-1,-1,29,2,5,1,-1,-1,93,31,3,22,-1,76,-1,69,31,-1,29,145,-1,23,56,-1,9,57,58,-1,-1,-1,24,24,-1,97,71,-1,85,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1

In [ ]:
train_b = train_b.loc[train_b['is_train'] == 1]

In [ ]:
train_target = pd.concat([train_target, train_b], ignore_index=True)
train_target.index.name = 'client_dk'

In [ ]:
train_target.to_csv(dis + 'predata_extended.csv')

## Function of Extend Data

In [ ]:
import progressbar
import time

def GetMoreTrain(data, train_target, l):
  col = train_target.columns[989:]
  train_b = train_target.drop(col, axis = 1)
  train_b_exp = train_target[col[199:]]

  data0 = data.loc[data['trans_date'] < l]

  d = [[] for i in range(199)]
  for i in range(199):
    c = data0.loc[data0['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
    for j in range(50000):
      if j in c.index:
        c0 = list(c[j].index)
        d[i].append((200 - c0[-1]))
      else:
        d[i].append(-1)
    bar.update((i + 1))

  for i in range(199):
    train_b['last_buy_small_group_' + str(i)] = d[i]

  bc = GetWeekly(data, 200)
  for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    bc[i][bc[i] > 0] = 1

  for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_b[str(i)] = bc[i]

  train_b = train_b.fillna(0)

  for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_b[str(i)] = train_b[str(i)].astype(int)

  train_b = train_b.merge(train_b_exp, on = 'client_dk', how='outer')

  train_b.index = list(map(lambda x: 'a' + str(x), train_b.index))

  train_b = train_b.loc[train_b['is_train'] == 1]

  train_target = pd.concat([train_target, train_b], ignore_index=True)
  train_target.index.name = 'client_dk'

  return train_target
  